In [2]:
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModels import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

#Clear the previous tensorflow graph
K.clear_session()

In [3]:
FilterN = 10
FilterL = 10

def RULmodel_SN_1(input_shape):
    
    print(input_shape)
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(250, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.6))
    model.add(Dense(50, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    #model.add(Dense(5, activation='relu', kernel_initializer='glorot_normal', name='fc3'))
    model.add(Dropout(0.2))
    #model.add(Dense(10, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='tanh', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model


def RULModel_CNN_1(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    print(input_shape)
    
    #Add the layers for the model
    model.add(Conv2D(1, (FilterN, FilterL), input_shape=input_shape, padding='same', kernel_initializer='glorot_normal', activation='relu', name='cl1'))
    model.add(Dropout(0.6))
    model.add(MaxPooling2D(pool_size=5, name='pl1'))
    model.add(Flatten())
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model


def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

In [4]:
#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel(selected_features, scaler = min_max_scaler, window_stride=2)
tModel.loadData()


In [16]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*tModel.windowSize
shapeCNN1 = (tModel.windowSize, nFeatures, 1)
shapeDCNN1 = (tModel.windowSize, nFeatures)

modelRULSN = RULmodel_SN_1(shapeSN)
modelRULCNN = RULModel_CNN_1(shapeCNN1)
DCNN = RULCNNModel(shapeDCNN1[0], shapeDCNN1[1])

modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
modelRULCNN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
DCNN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#Add the models to the tunableModel object
tModel.addModel('RULmodel_SN_1', modelRULSN)
tModel.addModel('RULModel_CNN_1', modelRULCNN)
tModel.addModel('RULCNNModel', DCNN)

420
(30, 14, 1)


C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C1", padding="same", kernel_initializer="glorot_normal", activation="tanh")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:62: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C2", padding="same", kernel_initializer="glorot_normal", activation="tanh")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C3", padding="same", kernel_initializer="glorot_normal", activation="tanh")`
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C4", padding="same", kernel_initializer="glorot_normal", acti

In [14]:
tModel.printData(printTop=False)

Printing shapes
Training data (X, y)
(5947, 420)
(5947, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing last 5 elements
Training data (X, y)
[[ 0.35211268  0.28537298  0.38511049 ...  0.         -0.36752137
  -0.39178241]
 [ 0.33802817  0.42447571  0.33190303 ...  0.6        -0.12820513
  -0.56018519]
 [ 0.47183099  0.19670826  0.18858614 ...  0.6        -0.28205128
  -0.5083912 ]
 [ 0.37323944  0.39102734  0.45977258 ...  0.         -1.
  -0.17476852]
 [ 0.12676056  0.18821343  0.74812272 ...  0.8        -0.48717949
  -0.92418981]]
[[14.]
 [11.]
 [ 8.]
 [ 5.]
 [ 2.]]
Testing data (X, y)
[[-0.50704225 -0.52641359 -0.55717657 ... -0.6         0.4017094
   0.6015625 ]
 [ 0.11267606 -0.1595434  -0.26625188 ...  0.2        -0.1965812
   0.12326389]
 [-0.04225352  0.14733209 -0.23192448 ...  0.          0.05982906
   0.31278935]
 [-0.33802817 -0.01406955 -0.57476936 ... -0.6         0.38461538
   0.30902778]
 [-0.21830986  0.07300239 -0.00064364 ...  0.4        -0.04273504
  -0.19444444

In [17]:
#Clear the previous tensorflow graph
K.clear_session()

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

#print(tModel.currentModelName)
#tModel.getAllModelsDescription()
#tModel.getModelDescription('RULModel_CNN_1')
#tModel.getModelDescription(tModel.currentModelName)
#tModel.getModelDescription()
tModel.setCurrentModel('RULmodel_SN_1')
tModel.getModelDescription()
tModel.trainCurrentModel(learningRateScheduler=lrate)


Description for current model: RULmodel_SN_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 250)               105250    
_________________________________________________________________
dropout_11 (Dropout)         (None, 250)               0         
_________________________________________________________________
fc2 (Dense)                  (None, 50)                12550     
_________________________________________________________________
dropout_12 (Dropout)         (None, 50)                0         
_________________________________________________________________
out (Dense)                  (None, 1)                 51        
Total params: 117,851
Trainable params: 117,851
Non-trainable params: 0
_________________________________________________________________
Epoch 1/250
5947/5947 [==============================] - 0s 50us/step - loss: 7171.8062 - m

5947/5947 [==============================] - 0s 8us/step - loss: 277.0689 - mean_squared_error: 277.0689
Epoch 131/250
5947/5947 [==============================] - 0s 9us/step - loss: 279.1731 - mean_squared_error: 279.1731
Epoch 132/250
5947/5947 [==============================] - 0s 8us/step - loss: 288.4199 - mean_squared_error: 288.4199
Epoch 133/250
5947/5947 [==============================] - 0s 9us/step - loss: 285.6209 - mean_squared_error: 285.6209
Epoch 134/250
5947/5947 [==============================] - 0s 9us/step - loss: 297.7175 - mean_squared_error: 297.7175
Epoch 135/250
5947/5947 [==============================] - 0s 8us/step - loss: 287.5547 - mean_squared_error: 287.5547
Epoch 136/250
5947/5947 [==============================] - 0s 9us/step - loss: 272.7767 - mean_squared_error: 272.7767
Epoch 137/250
5947/5947 [==============================] - 0s 8us/step - loss: 286.1232 - mean_squared_error: 286.1232
Epoch 138/250
5947/5947 [==============================] - 0s 

In [18]:
tModel.evaluateCurrentModel(["rhs"])
print("scores")

cScores = tModel.getModelScores(tModel.currentModelName)
rmse = math.sqrt(cScores['score_1'])
rhs = cScores['rhs']
time = tModel.getModelTimes(tModel.currentModelName)

print(cScores)
print("RMSE: {}".format(rmse))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 441us/step
scores
{'score_1': 221.4499005126953, 'loss': 221.4499005126953, 'rhs': 459.0884260633895}
RMSE: 14.88119284575989
RHS: 459.0884260633895
Time : 14.296709914386339 seconds
